In [46]:
source("../methods_extRemes.r")

fnm <- "eobs_tx1x-jul_weu_flattened.csv"

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

df <- read.csv(paste0("fitted-models/", fnm), skip = 2)
gmst <- read.csv("../90_data/gmst.csv")
df <- merge(gmst, df)

event_year <- max(df$year)
event_gmst <- gmst$gmst[gmst$year == event_year]
y <- which(df$year == event_year)

---
### **Fit GEV to each land surface cell**

In [47]:
# fit model to each grid cell (takes about 80s)
system.time({
    mle_gev = lapply(colnames(df)[-(1:2)], function(cnm) {
        
        if(sum(is.na(df[,cnm])) > 2) {
            return(NULL)
        } else {
            df_c = setNames(df[,c("gmst",cnm)], c("gmst", "tmax"))
            df_c = df_c[!is.na(df_c$tmax),]
            return(suppressWarnings(fevd(x=tmax, data=df_c, location.fun=~gmst, type="GEV", optim.args = list(method = "Nelder-Mead"))))
        }
    })
})

   user  system elapsed 
 78.635   0.064  78.819 

In [48]:
# parameter estimates, event value, return period
ml_res <- data.frame(t(do.call("rbind", lapply(1:length(mle_gev), function(i) {
    if(is.null(mle_gev[[i]])) {
        return(setNames(rep(NA,11), c("mu0", "mu1", "sigma0", "shape", "ep", "rp", "rl10", "rl20", "pr", "delta_I", "nll")))
    } else {
        mdl <- mle_gev[[i]]
        pars <- model_pars(mdl)
        qc = event_qcov(mdl, event_gmst)
        ev = mdl$x[y]
        
        ep = unname(1-pextRemes(mdl, q = ev, qcov = qc))
        rp = 1/(ep)
        rl10 = unname(return_level(mdl, 10, event_gmst))
        rl20 = unname(return_level(mdl, 20, event_gmst))
        
        pr = prob_ratio(mdl, ev, event_gmst, event_gmst - 1.2)
        
        nll = mdl$results$value
        
        if(is.finite(rp)) {
            di = delta_I(mdl, rp, event_gmst, event_gmst - 1.2)
        } else {
            di = NA
        }
        
        return(c("mu0" = unname(pars["mu0"]), "mu1" = unname(pars["mu1"]), "scale" = unname(pars["scale"]), "shape" = unname(pars["shape"]), 
                 "ep" = ep, "rp" = rp, "rl10" = rl10, "rl20" = rl20, "pr" = pr, "delta_I" = di, "nll" = nll))
    }
}))))
write.csv(ml_res, paste0("fitted-models/", gsub("flattened", "shift-ml-ests", fnm)))